<a href="https://colab.research.google.com/github/Soham109/Headstarter_Residence-Bonus_Project/blob/main/Headstarter_Bonus_Project_%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier

In [ ]:
train_df = pd.read_csv('fraudTrain.csv')
test_df = pd.read_csv('fraudTest.csv')

In [ ]:
train_df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:
target = train_df['is_fraud']
train_df = train_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'first', 'last', 'street',
                          'dob', 'trans_num','is_fraud'], axis=1)

In [ ]:
train_df = train_df.drop(['city', 'state', 'zip', 'job', 'unix_time'], axis=1)
train_df

,amt,gender,lat,long,city_pop,merch_lat,merch_long
0,4.97,F,36.0788,-81.1781,3495,36.011293,-82.048315
1,107.23,F,48.8878,-118.2105,149,49.159047,-118.186462
2,220.11,M,42.1808,-112.2620,4154,43.150704,-112.154481
3,45.00,M,46.2306,-112.1138,1939,47.034331,-112.561071
4,41.96,M,38.4207,-79.4629,99,38.674999,-78.632459
...,...,...,...,...,...,...,...
1296670,15.56,M,37.7175,-112.4777,258,36.841266,-111.690765
1296671,51.70,M,39.2667,-77.5101,100,38.906881,-78.246528
1296672,105.93,M,32.9396,-105.8189,899,33.619513,-105.130529
1296673,74.90,M,43.3526,-102.5411,1126,42.788940,-103.241160


In [ ]:
target

,is_fraud
0,0
1,0
2,0
3,0
4,0
...,...
1296670,0
1296671,0
1296672,0
1296673,0


In [ ]:
features = train_df
features

,amt,gender,lat,long,city_pop,merch_lat,merch_long
0,4.97,F,36.0788,-81.1781,3495,36.011293,-82.048315
1,107.23,F,48.8878,-118.2105,149,49.159047,-118.186462
2,220.11,M,42.1808,-112.2620,4154,43.150704,-112.154481
3,45.00,M,46.2306,-112.1138,1939,47.034331,-112.561071
4,41.96,M,38.4207,-79.4629,99,38.674999,-78.632459
...,...,...,...,...,...,...,...
1296670,15.56,M,37.7175,-112.4777,258,36.841266,-111.690765
1296671,51.70,M,39.2667,-77.5101,100,38.906881,-78.246528
1296672,105.93,M,32.9396,-105.8189,899,33.619513,-105.130529
1296673,74.90,M,43.3526,-102.5411,1126,42.788940,-103.241160


In [ ]:
features = pd.get_dummies(features, columns=['gender'])

In [ ]:
features.dropna()

,amt,lat,long,city_pop,merch_lat,merch_long,gender_F,gender_M
0,4.97,36.0788,-81.1781,3495,36.011293,-82.048315,True,False
1,107.23,48.8878,-118.2105,149,49.159047,-118.186462,True,False
2,220.11,42.1808,-112.2620,4154,43.150704,-112.154481,False,True
3,45.00,46.2306,-112.1138,1939,47.034331,-112.561071,False,True
4,41.96,38.4207,-79.4629,99,38.674999,-78.632459,False,True
...,...,...,...,...,...,...,...,...
1296670,15.56,37.7175,-112.4777,258,36.841266,-111.690765,False,True
1296671,51.70,39.2667,-77.5101,100,38.906881,-78.246528,False,True
1296672,105.93,32.9396,-105.8189,899,33.619513,-105.130529,False,True
1296673,74.90,43.3526,-102.5411,1126,42.788940,-103.241160,False,True


In [ ]:
features_test = test_df.drop('is_fraud', axis=1)
target_test = test_df['is_fraud']

In [ ]:
features_test = test_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'first', 'last', 'street',
                          'dob', 'trans_num','city', 'state', 'zip', 'job', 'unix_time'], axis=1)

In [ ]:
features_test.dropna()

,amt,gender,lat,long,city_pop,merch_lat,merch_long,is_fraud
0,2.86,M,33.9659,-80.9355,333497,33.986391,-81.200714,0
1,29.84,F,40.3207,-110.4360,302,39.450498,-109.960431,0
2,41.28,F,40.6729,-73.5365,34496,40.495810,-74.196111,0
3,60.05,M,28.5697,-80.8191,54767,28.812398,-80.883061,0
4,3.19,M,44.2529,-85.0170,1126,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...
555714,43.77,M,40.4931,-91.8912,519,39.946837,-91.333331,0
555715,111.84,M,29.0393,-95.4401,28739,29.661049,-96.186633,0
555716,86.88,F,46.1966,-118.9017,3684,46.658340,-119.715054,0
555717,7.99,M,44.6255,-116.4493,129,44.470525,-117.080888,0


In [ ]:
features_test = pd.get_dummies(features_test, columns=['gender'])

In [ ]:
features_test['PoptoAmtRatio'] = features_test['amt'] / features_test['city_pop']

In [ ]:
features['PoptoAmtRatio'] = features['amt'] / features['city_pop']

In [ ]:
features

,amt,lat,long,city_pop,merch_lat,merch_long,gender_F,gender_M,PoptoAmtRatio
0,4.97,36.0788,-81.1781,3495,36.011293,-82.048315,True,False,0.001422
1,107.23,48.8878,-118.2105,149,49.159047,-118.186462,True,False,0.719664
2,220.11,42.1808,-112.2620,4154,43.150704,-112.154481,False,True,0.052987
3,45.00,46.2306,-112.1138,1939,47.034331,-112.561071,False,True,0.023208
4,41.96,38.4207,-79.4629,99,38.674999,-78.632459,False,True,0.423838
...,...,...,...,...,...,...,...,...,...
1296670,15.56,37.7175,-112.4777,258,36.841266,-111.690765,False,True,0.060310
1296671,51.70,39.2667,-77.5101,100,38.906881,-78.246528,False,True,0.517000
1296672,105.93,32.9396,-105.8189,899,33.619513,-105.130529,False,True,0.117831
1296673,74.90,43.3526,-102.5411,1126,42.788940,-103.241160,False,True,0.066519


In [ ]:
features_test = features_test.drop('is_fraud', axis=1)

In [ ]:
X_train = features
y_train = target
X_test = features_test
y_test = target_test
X_train

,amt,lat,long,city_pop,merch_lat,merch_long,gender_F,gender_M,PoptoAmtRatio
0,4.97,36.0788,-81.1781,3495,36.011293,-82.048315,True,False,0.001422
1,107.23,48.8878,-118.2105,149,49.159047,-118.186462,True,False,0.719664
2,220.11,42.1808,-112.2620,4154,43.150704,-112.154481,False,True,0.052987
3,45.00,46.2306,-112.1138,1939,47.034331,-112.561071,False,True,0.023208
4,41.96,38.4207,-79.4629,99,38.674999,-78.632459,False,True,0.423838
...,...,...,...,...,...,...,...,...,...
1296670,15.56,37.7175,-112.4777,258,36.841266,-111.690765,False,True,0.060310
1296671,51.70,39.2667,-77.5101,100,38.906881,-78.246528,False,True,0.517000
1296672,105.93,32.9396,-105.8189,899,33.619513,-105.130529,False,True,0.117831
1296673,74.90,43.3526,-102.5411,1126,42.788940,-103.241160,False,True,0.066519


In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def evaluate_and_save_model(model, X_train, X_test, y_train, y_test, filename):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model.__class__.__name__} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, predictions))
    print("-------------")

    with open(filename, "wb") as file:
        pickle.dump(model, file)

    print(f"Model saved as {filename}")

In [ ]:
lr_model = LogisticRegression(random_state=42)
evaluate_and_save_model(lr_model, X_train, X_test, y_train, y_test, 'lr_model.pkl')

LogisticRegression Accuracy: 0.9955
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.00      0.00      0.00      2145

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       0.99      1.00      0.99    555719

-------------
Model saved as lr_model.pkl


In [ ]:
xgb_model = xgb.XGBClassifier(random_state=42)
evaluate_and_save_model(xgb_model, X_train, X_test, y_train, y_test, 'xgb_model.pkl')


XGBClassifier Accuracy: 0.9957
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.25      0.05      0.09      2145

    accuracy                           1.00    555719
   macro avg       0.62      0.53      0.54    555719
weighted avg       0.99      1.00      0.99    555719

-------------
Model saved as xgb_model.pkl


In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
evaluate_and_save_model(dt_classifier, X_train, X_test, y_train, y_test, 'dt_classifier')

DecisionTreeClassifier Accuracy: 0.9933
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.18      0.21      0.19      2145

    accuracy                           0.99    555719
   macro avg       0.59      0.60      0.60    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as dt_classifier


In [ ]:
nb_model = GaussianNB()
evaluate_and_save_model(nb_model, X_train, X_test, y_train, y_test, 'nb_model.pkl')

GaussianNB Accuracy: 0.9919
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    553574
           1       0.23      0.47      0.31      2145

    accuracy                           0.99    555719
   macro avg       0.61      0.73      0.65    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as nb_model.pkl


In [ ]:
feature_importances = xgb_model.feature_importances_
feature_names = features.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df

,Feature,Importance
0,amt,0.626656
6,gender_F,0.122311
3,city_pop,0.063036
2,long,0.062743
1,lat,0.058396
5,merch_long,0.033538
4,merch_lat,0.033320
7,gender_M,0.000000


In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
evaluate_and_save_model(xgb_model, X_train_resampled, X_test, y_train_resampled, y_test, 'xgb_model_smote.pkl')

XGBClassifier Accuracy: 0.9567
              precision    recall  f1-score   support

           0       1.00      0.96      0.98    553574
           1       0.06      0.68      0.11      2145

    accuracy                           0.96    555719
   macro avg       0.53      0.82      0.54    555719
weighted avg       1.00      0.96      0.97    555719

-------------
Model saved as xgb_model_smote.pkl


In [ ]:
evaluate_and_save_model(nb_model, X_train_resampled, X_test, y_train_resampled, y_test, 'nb_model_smote.pkl')

GaussianNB Accuracy: 0.9700
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    553574
           1       0.08      0.65      0.14      2145

    accuracy                           0.97    555719
   macro avg       0.54      0.81      0.56    555719
weighted avg       1.00      0.97      0.98    555719

-------------
Model saved as nb_model_smote.pkl


In [ ]:
evaluate_and_save_model(dt_classifier, X_train_resampled, X_test, y_train_resampled, y_test, 'dt_classifier_smote.pkl')

DecisionTreeClassifier Accuracy: 0.9862
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    553574
           1       0.11      0.36      0.17      2145

    accuracy                           0.99    555719
   macro avg       0.55      0.67      0.58    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as dt_classifier_smote.pkl


In [ ]:
evaluate_and_save_model(lr_model, X_train_resampled, X_test, y_train_resampled, y_test, 'lr_model_smote.pkl')

LogisticRegression Accuracy: 0.9527
              precision    recall  f1-score   support

           0       1.00      0.95      0.98    553574
           1       0.06      0.75      0.11      2145

    accuracy                           0.95    555719
   macro avg       0.53      0.85      0.54    555719
weighted avg       1.00      0.95      0.97    555719

-------------
Model saved as lr_model_smote.pkl


In [ ]:
import joblib
from sklearn.ensemble import VotingClassifier

xgb_model = joblib.load('xgb_model.pkl')
nb_model_smote = joblib.load('nb_model_smote.pkl')
dt_classifier_smote = joblib.load('dt_classifier_smote.pkl')
lr_model_smote = joblib.load('lr_model_smote.pkl')

voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb_model),
    ('nb', nb_model_smote),
    ('dt', dt_classifier_smote),
    ('lr', lr_model_smote)
], voting='hard')

In [ ]:
evaluate_and_save_model(voting_clf, X_train_resampled, X_test, y_train_resampled, y_test, 'voting_clf_v-hard.pkl')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier Accuracy: 0.9859
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    553574
           1       0.16      0.61      0.25      2145

    accuracy                           0.99    555719
   macro avg       0.58      0.80      0.62    555719
weighted avg       1.00      0.99      0.99    555719

-------------
Model saved as voting_clf_v-hard.pkl


In [ ]:
y_test

,is_fraud
0,0
1,0
2,0
3,0
4,0
...,...
555714,0
555715,0
555716,0
555717,0
